In [15]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json

with open('burglary.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

In [63]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'

In [73]:
params = {
    'offense_description':'AGGRAVATED BURGLARY',
    'incident reported': '01/01/2021T00:00:00'
}

In [74]:
response = requests.get(endpoint, params = params)

In [75]:
response 

<Response [400]>

In [70]:
res = response.json()

In [71]:
type(res)

list

In [72]:
res.keys()

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
max_diam = []
hazardous = []
miss_dist = []
for day, objs in res['message'].items():
    for obj in objs:
        max_diam.append(float(obj['estimated_diameter']['miles']['estimated_diameter_max']))
        hazardous.append(obj['is_potentially_hazardous_asteroid'])
        miss_dist.append(float(obj['close_approach_data'][0]['miss_distance']['miles']))
        
plt.figure(figsize = (17, 10))
plt.scatter(max_diam, miss_dist, c = hazardous)
plt.xlabel('max diameter (miles)')
plt.ylabel('miss distance (miles)');